In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
import math as mh

In [34]:
class Decision_Tree:          #Binary tree class implementation
    def __init__(self,entropy,gain,level,split_feature,class_name):
        self.entropy = entropy
        self.gain = gain
        self.level = level
        self.split_feature = split_feature
        self.class_name = class_name
        self.left = None
        self.right = None

def print_Tree(root,s):    #function to print the tree
    if root is None:
        return None
    print(s)
    print('Level: ',root.level)
    print('Entropy: ',root.entropy)
    print('Gain: ',root.gain)
    print('Split Feature: ',root.split_feature)
    print('Class Name: ',root.class_name)
    print()
    Print_Tree(root.left,"Left Tree")
    Print_Tree(root.right,"Right Tree")

In [25]:
def countSetosa(output):                                               # Function to get count of Setosa
    output = np.array(output[:])
    return (output == 0).sum()    

def countVersicolor(output):                                           # Function to get count of Versicolor
    output=np.array(output[:])
    return (output == 1).sum()
def countVirginica(output):
    output=np.array(output[:])
    return (output==2).sum()

In [26]:
def entropy(output):                                                
    no_of_setosa = countSetosa(output)                                   # Count the number of setosa out of total
    no_of_versicolor = countVersicolor(output)                           # Count the number of versicolor out of total
    no_of_virginica = countVirginica(output)                             # Count the number of virginica out of total
    total=no_of_setosa+no_of_versicolor+no_of_virginica
    
    p1 = no_of_setosa / total                                              # Calculate p1
    p2 = no_of_versicolor / total                                          # Calculate p2
    p3 = no_of_virginica / total                                           # Calculate p3 
    
    value = 0                                                            # Initialise value to be zero
    if p1 != 0:
        value += p1 * mh.log(p1,2)                                # Use formula of entropy to calculate entropy for 
    if p2 != 0:                                                          #  each class and keep adding them to value 
        value += p2 * mh.log(p2,2)
    if p3 != 0:
        value += p3 * mh.log(p3,2)
    if not value:                                               # If total entropy is zero, just return it
        return value
    value =- value                                         # Need to change the sign as per the formula
    return value

In [13]:
def splitGain(split1,split2,output):                                   # Function to get the split gain
    p1 = len(split1) / len(output)                                         # Calculate p1
    p2 = len(split2) / len(output)                                         # Calculate p2
    value = 0                                                     # Initialise value as 0
    if p1 != 0:
        value += p1 * mh.log(p1,2)
    if p2 != 0:
        value += p2 * mh.log(p2,2)
    return - value       

In [30]:
def gainRatio(f,i,data_node,output,features):                          # Function to obtain gain ratio
    no_of_features_left = len(features)
    #total_elements = len(data_node)
    no_of_setosa = countSetosa(data_node)                                # Count the number of setosa
    no_of_versicolor = countVersicolor(data_node)                        # Count the number of versicolor
    no_of_virginica = countVirginica(data_node)       # Count the number of virginica
    initial_entropy = entropy(output)                                    # Get initial entropy
    split1_data = data_node[data_node[f] > i]                              # x of split1
    split2_data = data_node[data_node[f] <= i]                             # x of split2
    split1_output = output[data_node[f] > i]                               # y of split1
    split2_output = output[data_node[f] <= i]                              # y of split1   
    final_entropy = 0                                                    # Initialise final entropy
    final_entropy += entropy(split1_output) * len(split1_output) / len(output)    # Add entropy of split1 to final entropy
    final_entropy += entropy(split2_output) * len(split2_output) / len(output)    # Add entropy of split2 to final entropy
    entropy_gain = initial_entropy - final_entropy                         # Calculate entropy gain
    split_gain = splitGain(split1_output,split2_output,output)           # Calculate split gain
    return entropy_gain / (split_gain)

In [8]:
def splitOn(f,data_node,output,features):                              # Function to decide feature to split upon
    lst = list((data_node[f]))
    temp_max = -1                                                        # Initialise temp_max
    imp = -1                                                             # Initialise variable imp which stores on which numeric value of a feature we have to split
    for i in range(len(lst) -1):                                        # Access elements of lst    
        s = lst[i]     
        e = lst[i + 1]
        mid = (s + e) / 2                                                    # Find mid of pairs (s,e)                                                  
        temp = gainRatio(f,mid,data_node,output,features)                # Get gain ratio for this particular split into temp
        if temp > temp_max:                                              # If temp is more than temp_max, update temp_max and imp
            temp_max = temp
            imp = mid
    
    return temp_max,imp

In [9]:
def FeatureSplitting(data_node,features,output):                       # Function to do splitting on features
    max_gain_ratio = 0                                                   # Initilialise the gain ratio
    feature_to_split_upon = None
    total_elements = len(data_node)
    for f in features:                                                 # Consider each feature
        temp,temp2 = splitOn(f,data_node,output,features)                # Get gain ratio and numeric value of feature to split upon
        if temp > max_gain_ratio:                                        # Update max_gain_ratio, feature_to_split_upon and imp if condition is true
            max_gain_ratio = temp
            feature_to_split_upon = f
            imp = temp2
    return feature_to_split_upon,max_gain_ratio,imp

In [31]:
def showDecisionTree(data_node,output,features,level):              # Function to show decision tree
    no_of_features_left = len(features)
    total_elements = len(data_node)
    no_of_setosa = countSetosa(output)                                # Count the number of setosa
    no_of_versicolor = countVersicolor(output)                        # Count the number of versicolor
    no_of_virginica = countVirginica(output)    # Count the number of virginica
    print('Level',level)                                  
    print('Count of setosa: ',no_of_setosa)
    print('Count of versicolor: ',no_of_versicolor)
    print('Count of virginica: ',no_of_virginica)
    print('Current Entropy is: ',entropy(output))                   # Calculate and print current entropy
    maximum = max(no_of_setosa,no_of_versicolor,no_of_virginica)
    if maximum == no_of_setosa:
        cls_name = 'Setosa'
    elif maximum == no_of_versicolor:
        cls_name = 'Versicolor'
    else :
        cls_name = 'Virginica'
    if no_of_setosa == total_elements or no_of_versicolor == total_elements or no_of_virginica == total_elements or no_of_features_left==0:
        root = Decision_Tree(entropy(output),level,"Can't split, reached leaf node",0,cls_name)
        print('Reached leaf node')  
        print()
        return root
    else:
        feature_to_split_upon,max_gain_ratio,imp = FeatureSplitting(data_node,features,output)
        print('Splitting on feature',feature_to_split_upon,'with gain ratio',max_gain_ratio)
        root = Decision_Tree(entropy(output),level,feature_to_split_upon,max_gain_ratio,cls_name)
        print()
        data_node1 = data_node[data_node[feature_to_split_upon] > imp]   # x part of splitted data(first split)
        data_node2 = data_node[data_node[feature_to_split_upon] <= imp]  # x part of splitted data(second split)
        output1 = output[data_node[feature_to_split_upon] > imp]         # y part of splitted data(first split)
        output2 = output[data_node[feature_to_split_upon] <= imp]        # y part of splitted data(second split)
        root.left = showDecisionTree(data_node1,output1,features,level + 1)        # Print next level for first split, add left child to root
        root.right = showDecisionTree(data_node2,output2,features,level + 1)       # Print next level for second split, add left child to root 
        return root

In [33]:
iris_data = pd.DataFrame(iris.data)                                                   # Make dataframe
iris_output = pd.DataFrame(iris.target)
iris_data.head()
iris_data.columns = iris.feature_names                                                # Update column values 
root = showDecisionTree(iris_data,iris_output,iris_data.columns,0)         

Level 0
Count of setosa:  50
Count of versicolor:  50
Count of virginica:  50
Current Entropy is:  1.584962500721156
Splitting on feature petal width (cm) with gain ratio 0.9999999999999999

Level 1
Count of setosa:  0
Count of versicolor:  50
Count of virginica:  50
Current Entropy is:  1.0
Splitting on feature petal width (cm) with gain ratio 0.6933647985912662

Level 2
Count of setosa:  0
Count of versicolor:  1
Count of virginica:  45
Current Entropy is:  0.15109697051711368
Splitting on feature petal length (cm) with gain ratio 0.2622302372762406

Level 3
Count of setosa:  0
Count of versicolor:  0
Count of virginica:  43
Current Entropy is:  0.0
Reached leaf node

Level 3
Count of setosa:  0
Count of versicolor:  1
Count of virginica:  2
Current Entropy is:  0.9182958340544896
Splitting on feature sepal width (cm) with gain ratio 1.0

Level 4
Count of setosa:  0
Count of versicolor:  1
Count of virginica:  0
Current Entropy is:  0.0
Reached leaf node

Level 4
Count of setosa:  0


<ipython-input-26-dc996d32116c>:7: RuntimeWarning: invalid value encountered in long_scalars
  p1 = no_of_setosa / total                                              # Calculate p1
<ipython-input-26-dc996d32116c>:8: RuntimeWarning: invalid value encountered in long_scalars
  p2 = no_of_versicolor / total                                          # Calculate p2
<ipython-input-26-dc996d32116c>:9: RuntimeWarning: invalid value encountered in long_scalars
  p3 = no_of_virginica / total                                           # Calculate p3
<ipython-input-26-dc996d32116c>:7: RuntimeWarning: invalid value encountered in long_scalars
  p1 = no_of_setosa / total                                              # Calculate p1
<ipython-input-26-dc996d32116c>:8: RuntimeWarning: invalid value encountered in long_scalars
  p2 = no_of_versicolor / total                                          # Calculate p2
<ipython-input-26-dc996d32116c>:9: RuntimeWarning: invalid value encountered in long_scalars
  

Splitting on feature petal length (cm) with gain ratio 0.6066178220203009

Level 3
Count of setosa:  0
Count of versicolor:  0
Count of virginica:  2
Current Entropy is:  0.0
Reached leaf node

Level 3
Count of setosa:  0
Count of versicolor:  49
Count of virginica:  3
Current Entropy is:  0.31821529768323314
Splitting on feature petal length (cm) with gain ratio 0.2720453440631925

Level 4
Count of setosa:  0
Count of versicolor:  2
Count of virginica:  2
Current Entropy is:  1.0
Splitting on feature petal width (cm) with gain ratio 1.0

Level 5
Count of setosa:  0
Count of versicolor:  2
Count of virginica:  0
Current Entropy is:  0.0
Reached leaf node

Level 5
Count of setosa:  0
Count of versicolor:  0
Count of virginica:  2
Current Entropy is:  0.0
Reached leaf node

Level 4
Count of setosa:  0
Count of versicolor:  47
Count of virginica:  1
Current Entropy is:  0.14609425012013633
Splitting on feature petal width (cm) with gain ratio 0.26298064861912657

Level 5
Count of setosa: 

In [35]:
print_Tree(root,"Root Node")

Root Node
Level:  petal width (cm)
Entropy:  1.584962500721156
Gain:  0
Split Feature:  0.9999999999999999
Class Name:  Setosa

Left Tree
Level:  petal width (cm)
Entropy:  1.0
Gain:  1
Split Feature:  0.6933647985912662
Class Name:  Versicolor

Left Tree
Level:  petal length (cm)
Entropy:  0.15109697051711368
Gain:  2
Split Feature:  0.2622302372762406
Class Name:  Virginica

Left Tree
Level:  Can't split, reached leaf node
Entropy:  0.0
Gain:  3
Split Feature:  0
Class Name:  Virginica

Right Tree
Level:  sepal width (cm)
Entropy:  0.9182958340544896
Gain:  3
Split Feature:  1.0
Class Name:  Virginica

Left Tree
Level:  Can't split, reached leaf node
Entropy:  0.0
Gain:  4
Split Feature:  0
Class Name:  Versicolor

Right Tree
Level:  Can't split, reached leaf node
Entropy:  0.0
Gain:  4
Split Feature:  0
Class Name:  Virginica

Right Tree
Level:  petal length (cm)
Entropy:  0.44506485705083865
Gain:  2
Split Feature:  0.6066178220203009
Class Name:  Versicolor

Left Tree
Level:  Can'